In [1]:
import glob
import json
from PIL import Image
import matplotlib.pyplot as plt

import torch
from torchvision import transforms

from azureml.core.webservice import Webservice
from azureml.core import Workspace
from azureml.core.model import Model

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azureml-telemetry 1.6.0 (c:\projects\repos\workshops\env\lib\site-packages), Requirement.parse('azureml-telemetry~=1.2.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.6.0.post1 (c:\projects\repos\workshops\env\lib\site-packages), Requirement.parse('azureml-core~=1.2.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (azureml-core 1.6.0.post1 (c:\projects\repos\workshops\env\lib\site-packages), Requirement.parse('azureml-core~=1.2.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.StepRun = azureml.pipeline.core.run:StepRun._from_dto with exc

In [2]:
ws = Workspace.from_config()
service_name = 'aci-cells-malaria'
service = Webservice(ws,service_name)

In [3]:
image_list_infected = list(glob.glob('../../Datasets/cell_images/test/infected/*.png'))[50]
image_list_uninfected = list(glob.glob('../../Datasets/cell_images/test/uninfected/*.png'))[50]

In [4]:
def preprocess(image_file):
    """Preprocess the input image."""
    data_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    image = Image.open(image_file)
    image = data_transforms(image).float()
    image = torch.tensor(image)
    image = image.unsqueeze(0)
    return image.numpy()

In [5]:
test_images = [{'label':'infected', 'data':preprocess(image)} for image in image_list_infected]
test_images += [{'label':'uninfected', 'data':preprocess(image)} for image in image_list_uninfected]

PermissionError: [Errno 13] Permission denied: '.'

In [ ]:
results = []
for input_data in test_images[0]:
    result = service.run(input_data=json.dumps({'data': input_data['data'].tolist()}))
    results.append(result)


In [ ]:
result

____

In [36]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient

from msrest.authentication import ApiKeyCredentials

# Now there is a trained endpoint that can be used to make a prediction
ENDPOINT = 'https://westus2.api.cognitive.microsoft.com/customvision/v3.0/Prediction/20725ff7-16ea-4c53-bb7b-548180c7f5ce/classify/iterations/Iteration1/image'
ENDPOINT = 'https://westus2.api.cognitive.microsoft.com/'
prediction_key = 'cb99d4e459a84ee4a7560c4eca6c47ac'
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)
publish_iteration_name = "Iteration1"
project_id = '20725ff7-16ea-4c53-bb7b-548180c7f5ce'
    

In [52]:

list_images = []

for path in image_list_infected:
    with open(path, "rb") as image_contents:
        list_images.append({'label':'infected','data':image_contents.read()})

for path in image_list_uninfected:
    with open(path, "rb") as image_contents:
        list_images.append({'label':'uninfected','data':image_contents.read()})

In [56]:
results_custom_vision = []

for i,image in enumerate(list_images):
    print(i,'/',len(list_images))

    predictions = predictor.classify_image(
        project_id, 
        'Iteration1', 
        image['data']
        ).predictions

    res = {predictions[0].tag_name:predictions[0].probability,predictions[1].tag_name:predictions[1].probability}
    results_custom_vision.append(
        {'label':image['label'],'infected_prob':res['infected'],'uninfected_prob':res['uninfected']}
        )

514
18 / 5514
19 / 5514
20 / 5514
21 / 5514
22 / 5514
23 / 5514
24 / 5514
25 / 5514
26 / 5514
27 / 5514
28 / 5514
29 / 5514
30 / 5514
31 / 5514
32 / 5514
33 / 5514
34 / 5514
35 / 5514
36 / 5514
37 / 5514
38 / 5514
39 / 5514
40 / 5514
41 / 5514
42 / 5514
43 / 5514
44 / 5514
45 / 5514
46 / 5514
47 / 5514
48 / 5514
49 / 5514
50 / 5514
51 / 5514
52 / 5514
53 / 5514
54 / 5514
55 / 5514
56 / 5514
57 / 5514
58 / 5514
59 / 5514
60 / 5514
61 / 5514
62 / 5514
63 / 5514
64 / 5514
65 / 5514
66 / 5514
67 / 5514
68 / 5514
69 / 5514
70 / 5514
71 / 5514
72 / 5514
73 / 5514
74 / 5514
75 / 5514
76 / 5514
77 / 5514
78 / 5514
79 / 5514
80 / 5514
81 / 5514
82 / 5514
83 / 5514
84 / 5514
85 / 5514
86 / 5514
87 / 5514
88 / 5514
89 / 5514
90 / 5514
91 / 5514
92 / 5514
93 / 5514
94 / 5514
95 / 5514
96 / 5514
97 / 5514
98 / 5514
99 / 5514
100 / 5514
101 / 5514
102 / 5514
103 / 5514
104 / 5514
105 / 5514
106 / 5514
107 / 5514
108 / 5514
109 / 5514
110 / 5514
111 / 5514
112 / 5514
113 / 5514
114 / 5514
115 / 5514
